In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pandas as pd
import os


from dash import Dash, dcc, html, Input, Output
import plotly.express as px



In [72]:
#df= pd.read_excel(os.path.join('..', '27_08_24.xlsx'))

path= [os.path.join('..', 'data', x) for x in os.listdir(os.path.join('..', 'data'))]
#df
df_0= pd.read_excel(path[0])
df_1= pd.read_excel(path[1])
df_2= pd.read_excel(path[2])
df_3= pd.read_excel(path[3])
df_4= pd.read_excel(path[4])
#df= pd.concat([df_0, df_1], )


In [74]:
df= pd.concat([df_0, df_1,df_2, df_3,df_4])

In [ ]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)


In [ ]:
df

In [11]:
df.columns

Index(['CONSOLIDADO(S)', 'ESPECIFICAÇÃO CRIME', 'MEIO EMPREGADO DEAC',
       'MUNICÍPIO(S)', 'BAIRRO(S)', 'ANO DO FATO', 'MÊS DO FATO', 'RISP',
       'AISP', 'DATA DO FATO', 'DIA DA SEMANA', 'HORA DO FATO',
       'FAIXA 4 HORAS', 'FAIXA 12 HORAS', 'LATITUDE', 'LONGITUDE', 'DISTRITO',
       'REGIÃO DE INTEGRAÇÃO', 'LOCAL DA OCORRÊNCIA', 'SEXO VÍTIMA',
       'IDADE VÍTIMA', 'FAIXA ETÁRIA VÍTIMA', 'TIPO DE VÍTIMA', 'COR VÍTIMA',
       'GRAU DE INSTRUÇÃO VÍTIMA', 'ESTADO CIVÍL VÍTIMA', 'SEXO AUTOR',
       'IDADE AUTOR', 'COR AUTOR', 'GRAU DE RELACIONAMENTO'],
      dtype='object')

In [ ]:
df.loc[df['CONSOLIDADO(S)'] == 'ROUBO', 'CONSOLIDADO(S)']

In [ ]:
dfteste= pd.DataFrame([df['CONSOLIDADO(S)'], df['MÊS DO FATO'], df['ANO DO FATO']]).T
#dfteste= dfteste.replace({'HOMICIDIO': 'MURDER'})
#dfteste= dfteste.replace({"JANEIRO":"JANUARY", "FEVEREIRO":"FEBRUARY", "MARÇO":"MARCH", "ABRIL":"APRIL", "MAIO":"MAY", "JUNHO":"JUNE", "JULHO":"JULY", "AGOSTO":"AUGUST", "SETEMBRO":"SEPTEMBER", "OUTUBRO":"OCTOBER", "NOVEMBRO":"NOVEMBER", "DEZEMBRO":"DECEMBER"})
#dfteste['YEAR']=  df['ANO DO FATO']

In [ ]:
dfteste= dfteste.groupby(['CONSOLIDADO(S)', 'MÊS DO FATO', 'ANO DO FATO']).size().reset_index(name='Contagem')


In [ ]:
dfteste

In [ ]:
#month_en=["JANUARY", "FEBRUARY", "MARCH", "APRIL", "MAY", "JUNE", "JULY", "AUGUST", "SEPTEMBER", "OCTOBER", "NOVEMBER", "DECEMBER"]

In [ ]:

app = Dash(__name__)

# Layout da aplicação
app.layout = html.Div([
    html.H1("Gráfico de Ocorrências por Mês e Ano"),
    
    # Dropdown para selecionar o tipo de crime
    dcc.Dropdown(
        id='tipo-crime-dropdown',
        options=[{'label': crime, 'value': crime} for crime in dfteste['CONSOLIDADO(S)'].unique()],
        value=dfteste['CONSOLIDADO(S)'].unique()[0],  # Valor inicial
        clearable=False
    ),
    
    # Componente para exibir o gráfico
    dcc.Graph(id='crime-graph'),

    # Anotação fora do gráfico
    html.Div("Fonte: Banco de Dados SEGUP", style={'textAlign': 'center', 'marginTop': '20px'})
])

# Callback para atualizar o gráfico com base na seleção do dropdown
@app.callback(
    Output('crime-graph', 'figure'),
    Input('tipo-crime-dropdown', 'value')
)
def update_graph(selected_crime):
    # Filtrar o DataFrame pelo tipo de crime selecionado
    df_filtered = dfteste[dfteste['CONSOLIDADO(S)'] == selected_crime]
    
    # Ordenar os meses
    meses_ordenados = ["JANEIRO", "FEVEREIRO", "MARÇO", "ABRIL", "MAIO", "JUNHO", 
                       "JULHO", "AGOSTO", "SETEMBRO", "OUTUBRO", "NOVEMBRO", "DEZEMBRO"]
    df_filtered['MÊS DO FATO'] = pd.Categorical(df_filtered['MÊS DO FATO'], categories=meses_ordenados, ordered=True)
    df_filtered = df_filtered.sort_values(['ANO DO FATO', 'MÊS DO FATO'])
    
    # Criar o gráfico de linhas
    fig = px.line(df_filtered, x='MÊS DO FATO', y='Contagem', color='ANO DO FATO', 
                  title=f'Ocorrências de {selected_crime} por Mês e Ano', markers=True)
    
    return fig

# Rodar a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
df_vulneravel= df[df['CONSOLIDADO(S)']=='ESTUPRO']
df_vulneravel= pd.concat([df_vulneravel, df[df['CONSOLIDADO(S)']=='ESTUPRO DE VULNERAVEL']]).dropna(subset='GRAU DE RELACIONAMENTO')
#[df['GRAU DE RELACIONAMENTO'].isin(['PADRASTO', 'PAI', 'TIO(A)', 'AVO', 'VIZINHO(A)'])]

df_vulneravel= df_vulneravel[df_vulneravel['GRAU DE RELACIONAMENTO']!='PREJUDICADO']
df_vulneravel= df_vulneravel[df_vulneravel['FAIXA ETÁRIA VÍTIMA']!='PREJUDICADO']


fig = px.sunburst(df_vulneravel, path=['FAIXA ETÁRIA VÍTIMA', 'GRAU DE RELACIONAMENTO' ] , title='Casos de estupro ocorridos no estado do Pará por grau de relacionamento', width=900, height=900)
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>Contagem de casos: %{value} <extra></extra>')
annotations = [
    dict(
        x=-0.1, 
        y=-0.08, 
        text="Os maiores números de casos são entre vítimas que tem menos de 17 anos<br>Os maiores números de casos são com pessoas próximas e conhecidas", 
        font=dict(size=15, color="black"),
        showarrow=False,
        bgcolor="lightgrey",  # Fundo cinza claro
        align='left'
    ),
]

fig.update_layout(annotations=annotations, title_font=dict(size=20) )

fig.show()
df_vulneravel

In [ ]:
df[df['CONSOLIDADO(S)']=='LESAO NO TRANSITO'].groupby('SEXO AUTOR').size()

In [75]:

df_filtered = df[df['MUNICÍPIO(S)'].isin(['BELEM', 'ANANINDEUA'])]

df_grouped = df_filtered.groupby(['MUNICÍPIO(S)', 'BAIRRO(S)']).size().reset_index(name='count')

df_top3 = df_grouped.sort_values(['MUNICÍPIO(S)', 'count'], ascending=False).groupby('MUNICÍPIO(S)').head(3)

fig = px.bar(df_top3, x='MUNICÍPIO(S)', y='count', color='BAIRRO(S)', barmode='group',
            title='Top 3 Bairros com Mais Incidentes em Belém e Ananindeua')
fig.show()

In [51]:
top_3_data_belem = df.groupby(['MUNICÍPIO(S)', 'BAIRRO(S)']).size()['BELEM'].sort_values(ascending=False).head(3)
#top_3_data_belem['MUNICÍPIO(S)']= ['BELEM', 'BELEM', 'BELEM', 'ANANINDEUA', 'ANANINDEUA', 'ANANINDEUA' ]
#Ananindeua

top_3_data_ananindeua = df.groupby(['MUNICÍPIO(S)', 'BAIRRO(S)']).size()['ANANINDEUA'].sort_values(ascending=False).head(3)
df_top=pd.concat([top_3_data_belem, top_3_data_ananindeua])
df_top

BAIRRO(S)
MARCO                   2232
PEDREIRA                1529
GUAMA                   1457
CIDADE NOVA             1485
CENTRO DE ANANINDEUA    1290
COQUEIRO ANANINDEUA      862
dtype: int64

In [59]:
ab= pd.DataFrame(df.groupby(['MUNICÍPIO(S)', 'BAIRRO(S)']).size())
ab[0]

MUNICÍPIO(S)     BAIRRO(S)                
ABAETETUBA       ABAETETUBA                   587
                 CENTRO DE ABAETETUBA         846
ABEL FIGUEIREDO  ABEL FIGUEIREDO               38
                 CENTRO DE ABEL FIGUEIREDO      6
ACARA            ACARA                        174
                                             ... 
XINGUARA         SELECTAS                       9
                 SETOR BELA VISTA               3
                 SETOR MARAJOA II               3
                 SETOR NOVO HORIZONTE           7
                 TANAKA                        17
Name: 0, Length: 566, dtype: int64

In [35]:

#Belem
top_3_data_belem = df.groupby(['MUNICÍPIO(S)', 'BAIRRO(S)']).size()['BELEM'].sort_values(ascending=False).head(3)

#Ananindeua

top_3_data_ananindeua = df.groupby(['MUNICÍPIO(S)', 'BAIRRO(S)']).size()['ANANINDEUA'].sort_values(ascending=False).head(3)
df_top= pd.concat([top_3_data_belem, top_3_data_ananindeua])
color_map = {
    'MARCO': '#636EFA',   # Substitua 'Bairro1_Belem' pelo nome real do bairro
    'PEDREIRA': '#EF553B',
    'CAMPINA': '#00CC96',
    'CIDADE NOVA': '#AB63FA',
    'CENTRO DE ANANINDEUA': '#19D3F3',
    'COQUEIRO ANANINDEUA': '#FFA15A'
}

appB = Dash(__name__)
appB.layout = html.Div([
    html.H4('Furtos por bairro em Belém e Ananindeua'),
    dcc.Graph(id="graph")
])
@appB.callback(
    Output("graph", "figure"), 
    Input("graph", "id"))
def update_bar_chart(_):
    
    fig = px.bar(df_top, x='MUNICÍPIO(S)',color='BAIRRO(S)',
 barmode="group",height=400)
    
    return fig

appB.run_server(debug=True)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[35], line 27, in update_bar_chart(_='graph')
     22 @appB.callback(
     23     Output("graph", "figure"), 
     24     Input("graph", "id"))
     25 def update_bar_chart(_):
---> 27     fig = px.bar(df_top, x='MUNICÍPIO(S)',color='BAIRRO(S)',
        df_top = BAIRRO(S)
MARCO                   2232
PEDREIRA                1529
GUAMA                   1457
CIDADE NOVA             1485
CENTRO DE ANANINDEUA    1290
COQUEIRO ANANINDEUA      862
dtype: int64
        px = <module 'plotly.express' from 'c:\\Users\\T-GAMER\\Desktop\\crime_analysis\\venv\\Lib\\site-packages\\plotly\\express\\__init__.py'>
     28  barmode="group",height=400)
     30     return fig

File c:\Users\T-GAMER\Desktop\crime_analysis\venv\Lib\site-packages\plotly\express\_chart_types.py:373, in bar(
    data_frame=BAIRRO(S)
MARCO                   2232
PEDREIRA

In [38]:
df.groupby(['MUNICÍPIO(S)', 'BAIRRO(S)']).size()['BELEM']

BAIRRO(S)
AEROPORTO         39
AGUA BOA         169
AGUAS NEGRAS     105
AGULHA           263
ARIRAMBA          48
                ... 
TERRA FIRME      562
UMARIZAL        1314
UNA               60
VAL-DE-CANS      264
VILA             102
Length: 68, dtype: int64

In [13]:
px.bar(df.iloc[:500], x='MÊS DO FATO')

In [17]:

appA = Dash(__name__)


appA.layout = html.Div([
    html.H4('Restaurant tips by day of week'),
    dcc.Dropdown(
        id="dropdown",
        options=["Fri", "Sat", "Sun"],
        value="Fri",
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])


@appA.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))
def update_bar_chart(day):
    df = px.data.tips() # replace with your own data source
    mask = df["day"] == day
    fig = px.bar(df[mask], x="sex", y="total_bill", 
                 color="smoker", barmode="group")
    return fig


appA.run_server(debug=True)